# Students markdown generator for academicpages

Takes a TSV of students with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `students.tsv` with one containing your data.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat students.tsv









02/02/2021	Beatrice Taylor	2021: Beatrice Taylor	UCL	ETA 2024	1-beatrice-taylor		Neil Oxtoby	Chris Hardy (DRC)	"Seb Crutch (DRC), Danny Alexander"

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [8]:
students = pd.read_csv("students.tsv", sep="\t", header=0)
students


,start_date,student_name,title,venue,thesis_title,url_slug,phd_url,primary,secondary,subsidiary
0,01/10/2014,Razvan Marinescu,2014: Razvan Marinescu,UCL,Modelling the Neuroanatomical Progression of A...,1-razvan-marinescu,https://discovery.ucl.ac.uk/id/eprint/10087438/,Danny Alexander,Seb Crutch (DRC),Neil Oxtoby
1,30/09/2016,Maura Bellio,2016: Maura Bellio,UCL,Translating Predictive Models for Alzheimer's ...,1-maura-bellio,https://discovery.ucl.ac.uk/id/eprint/10134507,Danny Alexander,Ann Blandford (UCLIC),"Neil Oxtoby, Annemie Ribbens (icometrix)"
2,01/10/2018,Anna Schroder,2018: Anna Schroder,UCL,A Framework for Pathology-Informed Optimisatio...,1-anna-schroder,NaN,Danny Alexander,Jonathan Schott (DRC),Neil Oxtoby
3,01/10/2019,Moucheng Xu,2019: Moucheng Xu,UCL,Visual and Causal Representation Learning for ...,1-moucheng-xu,NaN,Joe Jacob,Yipeng Hu (WEISS),"Marius de Groot (GSK), Neil Oxtoby"
4,01/09/2020,Kiran Samra,2020: Kiran Samra,UCL,Defining the stages of presymptomatic frontote...,1-kiran-samra,NaN,Jonathan Rohrer (DRC),Neil Oxtoby,Lucy Russell (DRC)
5,01/10/2020,Zeena Shawa,2020: Zeena Shawa,UCL,ETA 2024,2-zeena-shawa,NaN,Neil Oxtoby,Rimona Weil (DRC),Cameron Shand
6,01/12/2020,Isaac Llorente Saguer,2020: Isaac Llorente Saguer,UCL,ETA 2024,3-isaac-llorente-saguer,NaN,Neil Oxtoby,Marc Aurel Busche (UK DRI),NaN
7,02/02/2021,Beatrice Taylor,2021: Beatrice Taylor,UCL,ETA 2024,1-beatrice-taylor,NaN,Neil Oxtoby,Chris Hardy (DRC),"Seb Crutch (DRC), Danny Alexander"


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
import os
for row, item in students.iterrows():
    
    md_filename = str(item.start_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.start_date) + "-" + item.url_slug
    year = item.start_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: students"""
    
    md += """\npermalink: /students/""" + html_filename
    
    md += "\ndate: " + str(item.start_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
    
    md += f"\nStudent: {item.student_name}"
    
    if len(str(item.thesis_title)) > 5:
        md += "\n\nThesis: " + item.thesis_title + "<br/>"
    else:
        md += "\n\nThesis submission: " + item.thesis_title + "<br/>"
    
    if len(str(item.phd_url)) > 5:
        md += "\n[Download here](" + item.phd_url + ")"
    
    if len(str(item.primary)) > 5:
        md += "\n\nPrimary Supervisor: " + item.primary + "<br/>"
    if len(str(item.secondary)) > 5:
        md += "\nSecondary Supervisor: " + item.secondary + "<br/>"
    if len(str(item.subsidiary)) > 5:
        md += "\nSubsidiary: " + item.subsidiary
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_students/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [12]:
!ls ../_students/

2014-1-razvan-marinescu.md      2020-1-kiran-samra.md
2016-1-maura-bellio.md          2020-2-zeena-shawa.md
2018-1-anna-schroder.md         2020-3-isaac-llorente-saguer.md
2019-1-moucheng-xu.md           2021-1-beatrice-taylor.md


In [13]:
!cat ../_students/2014-1-razvan-marinescu.md

---
title: "2014: Razvan Marinescu"
collection: students
permalink: /students/01/10/2014-1-razvan-marinescu
date: 01/10/2014
venue: 'UCL'
---
Student: Razvan Marinescu

Thesis: Modelling the Neuroanatomical Progression of Alzheimer's Disease and Posterior Cortical Atrophy<br/>
[Download here](https://discovery.ucl.ac.uk/id/eprint/10087438/)

Primary Supervisor: Danny Alexander<br/>
Secondary Supervisor: Seb Crutch (DRC)<br/>
Subsidiary: Neil Oxtoby